Conhecendo a lib requests

In [4]:
import requests

In [12]:
r = requests.get('https://api.github.com/versions')

In [13]:
r.text

'["2022-11-28"]'

In [15]:
r.json()

['2022-11-28']

In [16]:
r = requests.get('https://api.github.com/events')

In [17]:
r.text

'[{"id":"40038351276","type":"PushEvent","actor":{"id":132768287,"login":"jurabek10","display_login":"jurabek10","gravatar_id":"","url":"https://api.github.com/users/jurabek10","avatar_url":"https://avatars.githubusercontent.com/u/132768287?"},"repo":{"id":825003327,"name":"jurabek10/burak-react","url":"https://api.github.com/repos/jurabek10/burak-react"},"payload":{"repository_id":825003327,"push_id":19261453025,"size":1,"distinct_size":1,"ref":"refs/heads/develop","head":"284f69db8a9c754f2910ae76fe5af6876ade150f","before":"b36bafe2c864204e5f78697169408b5d49ca6133","commits":[{"sha":"284f69db8a9c754f2910ae76fe5af6876ade150f","author":{"email":"samiev.jurabek@bk.ru","name":"jurabek10"},"message":"feat: develop header and footer part one","distinct":true,"url":"https://api.github.com/repos/jurabek10/burak-react/commits/284f69db8a9c754f2910ae76fe5af6876ade150f"}]},"public":true,"created_at":"2024-07-10T18:28:16Z"},{"id":"40038351287","type":"CreateEvent","actor":{"id":140785728,"login":"

In [21]:
headers = {'X-GitHub-Api-Version': '2022-11-28'}

Definindo a url base e owner de onde vamos extrair os dados

In [22]:
api_base_url = 'https://api.github.com'
owner = 'amzn'
url = f'{api_base_url}/users/{owner}/repos'

In [23]:
url

'https://api.github.com/users/amzn/repos'

In [24]:
response = requests.get(url, headers=headers)
response.status_code

200

In [26]:
len(response.json())

30

Configurando headers

In [1]:
access_token = 'your-api-key'
headers = {
  'Authorization': 'Bearer ' + access_token, 
  'X-GitHub-Api-Version': '2022-11-28'
}


Extraindo dados por paginação, obtendo nomes e linguagens dos repositórios

In [29]:
repos_list = []
for page_num in range(1, 6):
  try:
    url_page = f'{url}?page={page_num}'
    response = requests.get(url_page, headers=headers)
    repos_list.append(response.json())
  except:
    repos_list.append(None)

In [38]:
repos_list[0][0].get('name')

'.github'

In [50]:
repos_name = [repo.get('name') for page in repos_list for repo in page]

In [56]:
repos_name[:5]

['.github',
 'ads-advanced-tools-docs',
 'ads-pao-amznjs-gtm-template',
 'alexa-coho',
 'alexa-skills-kit-js']

In [57]:
len(repos_name)

150

In [60]:
repos_languages = [repo.get('language') for page in repos_list for repo in page]

In [61]:
repos_languages[:10]

[None,
 'Jupyter Notebook',
 'Smarty',
 'JavaScript',
 None,
 'Python',
 'PHP',
 'Java',
 'Python',
 'CSS']

In [62]:
len(repos_languages)

150

Criando um Dataframe

In [64]:
import pandas as pd

In [69]:
dados_amz = pd.DataFrame()
dados_amz['repository_name'] = repos_name
dados_amz['languages'] = repos_languages

In [66]:
dados_amz

,repository_name,languages
0,.github,None
1,ads-advanced-tools-docs,Jupyter Notebook
2,ads-pao-amznjs-gtm-template,Smarty
3,alexa-coho,JavaScript
4,alexa-skills-kit-js,None
...,...,...
145,workspace-resolver,JavaScript
146,ws-vae,Python
147,xfer,Python
148,zeek-plugin-bacnet,Zeek


Salvando o dataframe

In [67]:
dados_amz.to_csv('amazon.csv')

Criando repositório com POST

In [78]:
user_repo_url = f'{api_base_url}/user/repos'
user_repo_url

'https://api.github.com/user/repos'

In [79]:
data = {
  'name': 'linguagens-utilizadas',
  'description': 'Repositório com as linguagens da amazon',
  'private': False,
}

response = requests.post(user_repo_url, headers=headers, json=data)
response.url

'https://api.github.com/user/repos'

Formato do arquivo

In [80]:
import base64

In [81]:
with open('amazon.csv', 'rb') as file:
  file_content = file.read()

encoded_content = base64.b64encode(file_content)

In [83]:
encoded_content

b'LHJlcG9zaXRvcnlfbmFtZSxsYW5ndWFnZXMKMCwuZ2l0aHViLAoxLGFkcy1hZHZhbmNlZC10b29scy1kb2NzLEp1cHl0ZXIgTm90ZWJvb2sKMixhZHMtcGFvLWFtem5qcy1ndG0tdGVtcGxhdGUsU21hcnR5CjMsYWxleGEtY29obyxKYXZhU2NyaXB0CjQsYWxleGEtc2tpbGxzLWtpdC1qcywKNSxhbWF6b24tYWRzLWFkdmVydGlzZXItYXVkaWVuY2Utbm9ybWFsaXphdGlvbi1zZGstcHksUHl0aG9uCjYsYW1hem9uLWFkdmVydGlzaW5nLWFwaS1waHAtc2RrLFBIUAo3LGFtYXpvbi1jb2RlZ3VydS1wcm9maWxlci1mb3Itc3BhcmssSmF2YQo4LGFtYXpvbi1mcnVzdHJhdGlvbi1mcmVlLXNldHVwLWNlcnRpZmljYXRpb24tdG9vbCxQeXRob24KOSxhbWF6b24taHViLWNvdW50ZXItYXBpLWRvY3MsQ1NTCjEwLGFtYXpvbi1odWItY291bnRlci1hcGktc2FtcGxlcyxKYXZhCjExLGFtYXpvbi1odWItY291bnRlci1zZGstamF2YSxKYXZhCjEyLGFtYXpvbi1odWItc3VwcG9ydCxQb3dlclNoZWxsCjEzLGFtYXpvbi1pbnN0YW50LWFjY2Vzcy1zZGstamF2YSxKYXZhCjE0LGFtYXpvbi1pbnN0YW50LWFjY2Vzcy1zZGstbmV0LEMjCjE1LGFtYXpvbi1pbnN0YW50LWFjY2Vzcy1zZGstcGhwLFBIUAoxNixhbWF6b24taW5zdGFudC1hY2Nlc3Mtc2RrLXJ1YnksUnVieQoxNyxhbWF6b24tbGF1bmNoZXItaW50ZWdyYXRpb24tY29yZG92YS1wbHVnaW4sSmF2YVNjcmlwdAoxOCxhbWF6b24tbWFya2V0aW5nLXN0cmVhbS1leGFtcGxlcy

Upload de arquivo com PUT

In [2]:
username = 'your-username'
repo = 'linguagens-utilizadas'
path = 'amazon.csv'

url_update = f'{api_base_url}/repos/{username}/{repo}/contents/{path}'
url_update

NameError: name 'api_base_url' is not defined

In [85]:
data = {
  'message': 'add new archive',
  'content': encoded_content.decode('utf-8')
}

response = requests.put(url_update, json=data, headers=headers)
response.status_code

201